### Font type Recognition using Feed Forward Network

* <font size=4 color='green'>MSTC seminar on Deep Learning & Tensorflow</font>

In [ ]:
import tensorflow as tf
import numpy as np
import math


In [ ]:
!python -m pip install tqdm

In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
import time
for i in tqdm_notebook(range(100)):
    time.sleep(.1)

### Data: 2790 36x36 images

In [ ]:
# Load data
data = np.load('data_with_labels.npz')
train = data['arr_0']/255.
labels = data['arr_1']


In [ ]:
data['arr_0'].shape

In [ ]:
# Look at some data
print(train[0])
print(labels[0])

# If you have matplotlib installed
import matplotlib.pyplot as plt
%matplotlib inline

plt.ion()

In [ ]:
def to_onehot(labels,nclasses = 5):
    '''
    Convert labels to "one-hot" format.

    >>> a = [0,1,2,3]
    >>> to_onehot(a,5)
    array([[ 1.,  0.,  0.,  0.,  0.],
           [ 0.,  1.,  0.,  0.,  0.],
           [ 0.,  0.,  1.,  0.,  0.],
           [ 0.,  0.,  0.,  1.,  0.]])
    '''
    outlabels = np.zeros((len(labels),nclasses))
    for i,l in enumerate(labels):
        outlabels[i,l] = 1
    return outlabels

onehot = to_onehot(labels)

In [ ]:
# Split data into training and validation
indices = np.random.permutation(train.shape[0])
valid_cnt = int(train.shape[0] * 0.1)
test_idx,training_idx=indices[:valid_cnt],indices[valid_cnt:]
test, train = train[test_idx,:], train[training_idx,:]
onehot_test, onehot_train = onehot[test_idx,:], onehot[training_idx,:]

In [ ]:
sess = tf.InteractiveSession()

# These will be inputs
## Input pixels, flattened
x = tf.placeholder("float", [None, 1296])
## Known labels
y_ = tf.placeholder("float", [None,5])

# Hidden layer 1
num_hidden1 = 128
W1 = tf.Variable(tf.truncated_normal([1296,num_hidden1],
                               stddev=1./math.sqrt(1296)))
b1 = tf.Variable(tf.constant(0.1,shape=[num_hidden1]))
h1 = tf.sigmoid(tf.matmul(x,W1) + b1)

# Hidden Layer 2
num_hidden2 = 32
W2 = tf.Variable(tf.truncated_normal([num_hidden1,
            num_hidden2],stddev=2./math.sqrt(num_hidden1)))
b2 = tf.Variable(tf.constant(0.2,shape=[num_hidden2]))
h2 = tf.sigmoid(tf.matmul(h1,W2) + b2)

# Output Layer
W3 = tf.Variable(tf.truncated_normal([num_hidden2, 5],
                                   stddev=1./math.sqrt(5)))
b3 = tf.Variable(tf.constant(0.1,shape=[5]))

# Just initialize
sess.run(tf.initialize_all_variables())

# Define model
y = tf.matmul(h2,W3) + b3

### End model specification, begin training code

#predictions for our confusion matrix
preds=tf.nn.softmax(y)

In [ ]:
# Climb on cross-entropy
cross_entropy = tf.reduce_mean(
     tf.nn.softmax_cross_entropy_with_logits(y + 1e-50, y_))

# How we train
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Define accuracy
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
# Actually train
epochs = 1000
train_acc = np.zeros(epochs//10)
test_acc = np.zeros(epochs//10)
for i in tqdm(range(epochs), ascii=True):
    if i % 10 == 0:  # Record summary data, and the accuracy
        # Check accuracy on train set
        A = accuracy.eval(feed_dict={x: train.reshape([-1,1296]), y_: onehot_train})
        train_acc[i//10] = A

        # And now the validation set
        A = accuracy.eval(feed_dict={x: test.reshape([-1,1296]), y_: onehot_test})
        test_acc[i//10] = A

    train_step.run(feed_dict={x: train.reshape([-1,1296]), y_: onehot_train})

dnn_test_pred=sess.run(preds,feed_dict={x: test.reshape([-1,1296]), y_: onehot_test})

In [ ]:
# Check accuracy on train set

A = accuracy.eval(feed_dict={x: train.reshape([-1,1296]), y_: onehot_train})
print 'Training Accuracy= %s' % A

In [ ]:
# And now the validation set
A = accuracy.eval(feed_dict={x: test.reshape([-1,1296]), y_: onehot_test})
print 'Test Accuracy= %s' % A

In [ ]:
# Plot the accuracy curves
plt.plot(train_acc,'b')
plt.plot(test_acc,'r')

In [ ]:
# Look at the final testing confusion matrix
pred = np.argmax(y.eval(feed_dict={x: test.reshape([-1,1296]), y_: onehot_test}), axis = 1)
conf = np.zeros([5,5])
for p,t in zip(pred,np.argmax(onehot_test,axis=1)):
    conf[t,p] += 1

plt.matshow(conf)
plt.colorbar()

In [ ]:
# Let's look at a subplot of some weights
f, plts = plt.subplots(4,8, sharex=True)
for i in range(32):
    plts[i//8, i%8].matshow(W1.eval()[:,i].reshape([36,36]))

In [ ]:
# Examine the output weights
plt.matshow(W3.eval())
plt.colorbar()

In [ ]:
sess.close()

<h2>Confusion Matrix</h2>

In [ ]:
#packages for confusion matrix
import itertools
import random
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.ticker as ticker
%matplotlib inline

In [ ]:
a=[]
b=[]

for i in dnn_test_pred:
    m=np.argmax(i)
    a.append(m)
    
for j in onehot_test:
    n=np.argmax(j)
    b.append(n)
    
    
prediction=np.asarray(a)
label=np.asarray(b)

In [ ]:
class_names=['0' ,'1' ,'2' ,'3','4'] 


#Definition of our plotting
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    
    
    """
    This function prints and plots the confusion matrix (cm).
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation='horizontal')
    plt.yticks(tick_marks, classes)
    
    
#Set differences with and without normalization

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
  
 #some slight changes in order to plot % in the normalize confusion matrix
    if normalize:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, (("%.1f" % (cm[i,j]*100))),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        
    else:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(label, prediction)

#Setting print options. Float numbers precision of 2 for our output.
np.set_printoptions(precision=2) 

In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix %')

plt.show()